In [2]:
# Welcome to your new notebook
# Type here in the cell editor to add code!
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *

StatementMeta(, 6c2a806e-c8b5-4bc7-a384-2cc9a7afc323, 4, Finished, Available, Finished)

In [3]:
%%sql

create table if not exists Gold_OrderReturn
(
    
Order_ID  string,
Return string,
Order_Year int,
Order_Month int,
Created_TS TIMESTAMP,
Modified_TS TIMESTAMP

)
using delta 
partitioned by(Order_Year,Order_Month)

StatementMeta(, 6c2a806e-c8b5-4bc7-a384-2cc9a7afc323, 5, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

Now we will implement the incremental load to pull the data from the bronze table

In [4]:
# to handle null value we use coalesce function
Max_Date = spark.sql("select coalesce(max(Modified_TS),'1900-01-01') from Gold_OrderReturn").first()[0]

StatementMeta(, 6c2a806e-c8b5-4bc7-a384-2cc9a7afc323, 6, Finished, Available, Finished)

In [5]:
df=spark.sql("""select Order_ID  ,
Return ,
Order_Year, 
Order_Month ,
Created_TS ,
Modified_TS 
from bronze_sales where Return="Yes" and Modified_TS>'{}'""".format(Max_Date)) #pulling data from bronze using incremental logic

StatementMeta(, 6c2a806e-c8b5-4bc7-a384-2cc9a7afc323, 7, Finished, Available, Finished)

In [6]:
df.createOrReplaceTempView("ViewReturn")

StatementMeta(, 6c2a806e-c8b5-4bc7-a384-2cc9a7afc323, 8, Finished, Available, Finished)

In [9]:
%%sql
select * from ViewReturn

StatementMeta(, 6c2a806e-c8b5-4bc7-a384-2cc9a7afc323, 11, Finished, Available, Finished)

<Spark SQL result set with 123 rows and 6 fields>

Now inserting data into gold return table

In [10]:
%%sql
insert into Gold_OrderReturn --It’s equivalent to .write.mode("append") in PySpark.
select * from ViewReturn

StatementMeta(, 6c2a806e-c8b5-4bc7-a384-2cc9a7afc323, 12, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

In [11]:
%%sql
select * from Gold_OrderReturn

StatementMeta(, 6c2a806e-c8b5-4bc7-a384-2cc9a7afc323, 13, Finished, Available, Finished)

<Spark SQL result set with 123 rows and 6 fields>